In [ ]:
import pandas as pd
import pickle
import  category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from category_encoders import *
import  numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#pip install category_encoders

In [ ]:
url = "https://raw.githubusercontent.com/joao-l-rossi/LHL---midterm/main/mid-term-project-I/data/Flights_TRAIN.csv"
dataset = pd.read_csv(url)
dataset.isnull().sum()
dataset = dataset.dropna()
dataset = dataset.sample(frac=1, random_state=1234).reset_index(drop=True)
dataset.shape

(143046, 29)

In [ ]:
cols_to_encode = ["origin", "dest","op_unique_carrier"]
cols_X = ['dep_dayofweek','op_unique_carrier', 'origin', 'dest',
       'arr_Fog', 'arr_Snow', 'arr_Cold', 'arr_Storm', 'arr_Hail',
          'arr_Precipitation', 'crs_dep_time', 'crs_arr_time',
          'flights_origin', 'flights_dest']
#pd.get_dummies(dataset["op_unique_carrier"])

In [ ]:
X = dataset[cols_X]
y = dataset["arr_delay"]
X.head()

,dep_dayofweek,op_unique_carrier,origin,dest,arr_Fog,arr_Snow,arr_Cold,arr_Storm,arr_Hail,arr_Precipitation,crs_dep_time,crs_arr_time,flights_origin,flights_dest
0,0,EV,ORD,CVG,1.0,3.0,0.0,0.0,0.0,0.0,1135,1355,250,33
1,0,AA,EWR,PHX,1.0,3.0,0.0,0.0,0.0,0.0,955,1340,90,120
2,4,DL,SLC,CVG,1.0,3.0,0.0,0.0,0.0,0.0,1710,2235,78,16
3,1,F9,TPA,DEN,1.0,3.0,0.0,0.0,0.0,0.0,1439,1654,68,178
4,2,MQ,CID,ORD,1.0,1.0,0.0,0.0,0.0,0.0,1810,1929,6,296


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1234)

In [ ]:
X_train = X_train.reset_index(drop=True)     
y_train = y_train.reset_index(drop=True)

In [ ]:
encoder = TargetEncoder(cols=cols_to_encode)

In [ ]:
X_train_fitted = X_train.iloc[0:0,:]

In [ ]:
kf = KFold(n_splits = 5, shuffle=False)
for tr_ind, val_ind in kf.split(X_train, y_train):
    encoder.fit(X_train.loc[tr_ind], y_train.loc[tr_ind])
    X_train_fitted = X_train_fitted.append(encoder.transform(X_train.loc[val_ind]))

In [ ]:
encoder.fit(X_train, y_train)
X_test_fitted = encoder.transform(X_test)

Baseline

In [ ]:
mod = RandomForestRegressor()
mod.fit(X_train_fitted,y_train)

RandomForestRegressor()

In [ ]:
print(f"R2: {mod.score(X_test_fitted,y_test)}")
y_pred = mod.predict(X_test_fitted)
print(f"MSE: {metrics.mean_squared_error(y_test,y_pred)}")

R2: 0.10428110810865943
MSE: 1940.609101377141


In [ ]:
# params = {'bootstrap': [True, False],
#           'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#           'max_features': ['auto', 'sqrt'],
#           'min_samples_leaf': [1, 2, 4],
#           'min_samples_split': [2, 5, 10],
#           'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [ ]:
params = {
    'max_depth': [10, 15, 20],
    'n_estimators': [200,300,400]

}

In [ ]:
param_comb = 4

In [ ]:
mod = RandomForestRegressor()

In [ ]:
random_search = RandomizedSearchCV(mod, param_distributions=params, n_iter=param_comb, n_jobs=-1, verbose=10, random_state=1234, cv = kf.split(X_train, y_train))
random_search.fit(X_train_fitted, y_train)
print(random_search.best_score_)
print(random_search.best_estimator_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.08195512427585813
RandomForestRegressor(max_depth=10, n_estimators=400)


In [ ]:
best_est = random_search.best_estimator_
#best_est.fit(X_train_fitted,y_train)

RandomForestRegressor(max_depth=10, n_estimators=50)

In [ ]:
print(f"R2: {best_est.score(X_test_fitted,y_test)}")
y_pred = best_est.predict(X_test_fitted)
print(f"MSE: {metrics.mean_squared_error(y_test,y_pred)}")

R2: 0.11145487959146039
MSE: 1925.066852177364


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_save_name = 'inalized_model_Joao.sav'
path = F"/content/gdrive/My Drive/MidtermLHL/{model_save_name}"
pickle.dump(best_est, open(path, 'wb'))

In [ ]:
random_search.__dict__

{'best_estimator_': RandomForestRegressor(max_depth=10, n_estimators=200),
 'best_index_': 3,
 'best_params_': {'max_depth': 10, 'n_estimators': 200},
 'best_score_': 0.08239109921906274,
 'cv': <generator object _BaseKFold.split at 0x7f2b46daebd0>,
 'cv_results_': {'mean_fit_time': array([107.88637066, 212.55001445, 211.9552196 ,  92.07976947]),
  'mean_score_time': array([1.45665793, 2.70848408, 2.76706729, 0.55892606]),
  'mean_test_score': array([0.0601931 , 0.06852707, 0.06766781, 0.0823911 ]),
  'param_max_depth': masked_array(data=[50, 50, None, 10],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'param_n_estimators': masked_array(data=[100, 200, 200, 200],
               mask=[False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_depth': 50, 'n_estimators': 100},
   {'max_depth': 50, 'n_estimators': 200},
   {'max_depth': None, 'n_estimators': 200},
   {'max_depth': 10, 'n